# Analyze the baseline results
We run word-level RoBERTa coreference model on the movie scripts, on both the dev and train set. \
The model has been trained on OntoNotes v5 dataset.

In [1]:
# Import modules
import pandas as pd
import numpy as np
import collections
import os
import tqdm
import scipy.stats

pd.set_option("display.max_columns", 100)

In [2]:
# Read train and dev baseline csv files
train_baseline_df = pd.read_csv(
    os.path.join(os.getenv("DATA_DIR"),
                 "mica_text_coref/movie_coref/results/coreference/baselines/train.genre_wb.baseline.tsv"),
    sep="\t", index_col=None)
dev_baseline_df = pd.read_csv(
    os.path.join(os.getenv("DATA_DIR"), "mica_text_coref/movie_coref/results/coreference/baselines/dev.baseline.tsv"),
    sep="\t", index_col=None)

In [3]:
# Summarize baseline dataframes
print("train baseline (6 full-length screenplays):")
print(f"columns = \n{train_baseline_df.columns.tolist()}")
print(f"shape = {train_baseline_df.shape}")
for column, dtype in zip(train_baseline_df.columns, train_baseline_df.dtypes):
    if dtype != float:
        unique_vals = train_baseline_df[column].unique().tolist()
        print(f"{column:25s}: {unique_vals}")
print()

print("dev baseline (3 short script excerpts)")
print(f"columns = \n{dev_baseline_df.columns.tolist()}")
print(f"shape = {dev_baseline_df.shape}")
for column, dtype in zip(dev_baseline_df.columns, dev_baseline_df.dtypes):
    if dtype != float:
        unique_vals = dev_baseline_df[column].unique().tolist()
        print(f"{column:25s}: {unique_vals}")

train baseline (6 full-length screenplays):
columns = 
['preprocess', 'genre', 'entity', 'merge_speakers', 'provide_gold_mentions', 'remove_gold_singletons', 'split_len', 'overlap_len', 'merge_strategy', 'movie', 'metric', 'P', 'R', 'F']
shape = (108864, 14)
preprocess               : ['addsays', 'nocharacters', 'none']
genre                    : ['wb']
entity                   : ['all', 'person', 'speaker']
merge_speakers           : [False, True]
provide_gold_mentions    : [False, True]
remove_gold_singletons   : [False, True]
split_len                : [2048, 3072, 4096, 5120]
overlap_len              : [128, 256, 512]
merge_strategy           : ['after', 'average', 'before', 'max', 'min', 'none']
movie                    : ['avengers_endgame', 'dead_poets_society', 'john_wick', 'prestige', 'quiet_place', 'zootopia', 'all']
metric                   : ['muc', 'bcub', 'ceafe']

dev baseline (3 short script excerpts)
columns = 
['preprocess', 'genre', 'entity', 'merge_speakers', 'provi

In [96]:
# Find the performance for each hyperparam setting and the best performing hyperparameter for the train baseline
train_pred_baseline_df = train_baseline_df[~train_baseline_df["provide_gold_mentions"]
                                           & ~train_baseline_df["remove_gold_singletons"]]
hyperparams = ["preprocess", "genre", "entity", "merge_speakers", "provide_gold_mentions",
               "remove_gold_singletons", "split_len", "overlap_len", "merge_strategy"]
header = ([f"macro_{metric}_{score}_{agg}" for metric in ["muc", "bcub", "ceafe"]
                                          for score in ["p", "r", "f"]
                                          for agg in ["mean", "std"]]
          + ["macro_conll_f_mean", "macro_conll_f_std"]
          + [f"micro_{metric}_{score}" for metric in ["muc", "bcub", "ceafe"]
                                     for score in ["p", "r", "f"]]
          + ["micro_conll_f"])
rows = []
groups = train_pred_baseline_df.groupby(hyperparams)
for hyperparam_values, hyperparam_df in tqdm.tqdm(groups, total=groups.ngroups, unit="hyperparam"):
    row = list(hyperparam_values)

    # Macro value
    movie_muc_df = hyperparam_df.loc[(hyperparam_df["movie"] != "all")
                                     & (hyperparam_df["metric"] == "muc"), ["P", "R", "F"]]
    movie_bcub_df = hyperparam_df.loc[(hyperparam_df["movie"] != "all")
                                       & (hyperparam_df["metric"] == "bcub"), ["P", "R", "F"]]
    movie_ceafe_df = hyperparam_df.loc[(hyperparam_df["movie"] != "all")
                                        & (hyperparam_df["metric"] == "ceafe"), ["P", "R", "F"]]
    movie_conll_F1 = (hyperparam_df.loc[hyperparam_df["movie"] != "all", ["movie", "F"]].groupby("movie").mean()
                                  .agg(["mean", "std"]).values.flatten().tolist())
    row += movie_muc_df.agg(["mean", "std"]).values.T.flatten().tolist()
    row += movie_bcub_df.agg(["mean", "std"]).values.T.flatten().tolist()
    row += movie_ceafe_df.agg(["mean", "std"]).values.T.flatten().tolist()
    row += movie_conll_F1

    # Micro value
    row += hyperparam_df.loc[hyperparam_df["movie"] == "all", ["P", "R", "F"]].values.flatten().tolist()
    row.append(hyperparam_df.loc[hyperparam_df["movie"] == "all", "F"].mean())
    rows.append(row)

# Create column index
index = pd.MultiIndex.from_product([["macro"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["mean", "std"]],
                                   sortorder=None)
index = index.union(pd.MultiIndex.from_tuples([("macro", "conll", "f", "mean"), ("macro", "conll", "f", "std")],
                                              sortorder=None), sort=False)
index = index.union(pd.MultiIndex.from_product([["micro"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["value"]],
                                              sortorder=None), sort=False)
index = index.union(pd.MultiIndex.from_tuples([("micro", "conll", "f", "value")]), sort=False)
index.names = ["agg", "metric", "score", "func"]

# Create dataframe for train baseline
train_hyperparam_df = pd.DataFrame(rows, columns=hyperparams + header)
train_hyperparam_df = train_hyperparam_df.set_index(hyperparams)
train_hyperparam_df.columns = index

display(train_hyperparam_df)

100%|██████████| 1296/1296 [00:20<00:00, 62.92hyperparam/s]


agg                                                                                                                            macro  \
metric                                                                                                                           muc   
score                                                                                                                              p   
func                                                                                                                            mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy              
addsays    wb    all     False          False                 False                  2048      128         after           84.704428   
                                                                                                           average         84.686564   
                                                                                                           before          84.686564   
                                                                                                           max             84.718997   
                                                                                                           min             84.678523   
...                                                                                                                              ...   
none       wb    speaker True           False                 False                  5120      512         average         78.925721   
                                                                                                           before          78.925721   
                                                                                                           max             78.951570   
                                                                                                           min             78.884879   
                                                                                                           none            78.868407   

agg                                                                                                                                  \
metric                                                                                                                                
score                                                                                                                                 
func                                                                                                                            std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy             
addsays    wb    all     False          False                 False                  2048      128         after           3.510968   
                                                                                                           average         3.508866   
                                                                                                           before          3.508866   
                                                                                                           max             3.498933   
                                                                                                           min             3.518965   
...                                                                                                                             ...   
none       wb    speaker True           False                 False                  5120      512         average         7.294623   
                                                                                                           before          7.294623   
                                                                    

In [104]:
# best baseline performance on train set
train_hyperparam_df.loc[train_hyperparam_df["macro"]["conll"]["f"]["mean"] == (
                        train_hyperparam_df["macro"]["conll"]["f"]["mean"].max()),
                        ("macro", slice(None), "f")]

agg                                                                                                                            macro  \
metric                                                                                                                           muc   
score                                                                                                                              f   
func                                                                                                                            mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy              
addsays    wb    speaker True           False                 False                  3072      512         min             84.057317   

agg                                                                                                                                  \
metric                                                                                                                                
score                                                                                                                                 
func                                                                                                                            std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy             
addsays    wb    speaker True           False                 False                  3072      512         min             4.080106   

agg                                                                                                                                   \
metric                                                                                                                          bcub   
score                                                                                                                              f   
func                                                                                                                            mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy              
addsays    wb    speaker True           False                 False                  3072      512         min             59.603098   

agg                                                                                                                                  \
metric                                                                                                                                
score                                                                                                                                 
func                                                                                                                            std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy             
addsays    wb    speaker True           False                 False                  3072      512         min             6.616547   

agg                                                                                                                                   \
metric                                                                                                                         ceafe   
score                                                                                                                              f   
func                                                                                                                            mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons split_len overlap_len merge_strategy              
addsays    wb    speaker True           False                 Fa

The baseline achieves 64.36 conll F1 on train set

In [98]:
dev_pred_baseline_df = dev_baseline_df[~dev_baseline_df["provide_gold_mentions"]
                                       & ~dev_baseline_df["remove_gold_singletons"]]
hyperparams = ["preprocess", "genre", "entity", "merge_speakers", "provide_gold_mentions", "remove_gold_singletons"]
header = ([f"macro_{metric}_{score}_{agg}" for metric in ["muc", "bcub", "ceafe"]
                                          for score in ["p", "r", "f"]
                                          for agg in ["mean", "std"]]
          + ["macro_conll_f_mean", "macro_conll_f_std"]
          + [f"micro_{metric}_{score}" for metric in ["muc", "bcub", "ceafe"]
                                     for score in ["p", "r", "f"]]
          + ["micro_conll_f"])
rows = []
groups = dev_pred_baseline_df.groupby(hyperparams)
for hyperparam_values, hyperparam_df in tqdm.tqdm(groups, total=groups.ngroups, unit="hyperparam"):
    row = list(hyperparam_values)

    # Macro value
    movie_muc_df = hyperparam_df.loc[(hyperparam_df["movie"] != "all")
                                     & (hyperparam_df["metric"] == "muc"), ["precision", "recall", "f1"]]
    movie_bcub_df = hyperparam_df.loc[(hyperparam_df["movie"] != "all")
                                       & (hyperparam_df["metric"] == "bcub"), ["precision", "recall", "f1"]]
    movie_ceafe_df = hyperparam_df.loc[(hyperparam_df["movie"] != "all")
                                        & (hyperparam_df["metric"] == "ceafe"), ["precision", "recall", "f1"]]
    movie_conll_F1 = (hyperparam_df.loc[hyperparam_df["movie"] != "all", ["movie", "f1"]].groupby("movie").mean()
                                   .agg(["mean", "std"]).values.flatten().tolist())
    row += movie_muc_df.agg(["mean", "std"]).values.T.flatten().tolist()
    row += movie_bcub_df.agg(["mean", "std"]).values.T.flatten().tolist()
    row += movie_ceafe_df.agg(["mean", "std"]).values.T.flatten().tolist()
    row += movie_conll_F1

    # Micro value
    row += hyperparam_df.loc[hyperparam_df["movie"] == "all", ["precision", "recall", "f1"]].values.flatten().tolist()
    row.append(hyperparam_df.loc[hyperparam_df["movie"] == "all", "f1"].mean())
    rows.append(row)

# Create column index
index = pd.MultiIndex.from_product([["macro"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["mean", "std"]],
                                   sortorder=None)
index = index.union(pd.MultiIndex.from_tuples([("macro", "conll", "f", "mean"), ("macro", "conll", "f", "std")],
                                              sortorder=None), sort=False)
index = index.union(pd.MultiIndex.from_product([["micro"], ["muc", "bcub", "ceafe"], ["p", "r", "f"], ["value"]],
                                              sortorder=None), sort=False)
index = index.union(pd.MultiIndex.from_tuples([("micro", "conll", "f", "value")]), sort=False)
index.names = ["agg", "metric", "score", "func"]

# Create dataframe for dev baseline
dev_hyperparam_df = pd.DataFrame(rows, columns=hyperparams + header)
dev_hyperparam_df = dev_hyperparam_df.set_index(hyperparams)
dev_hyperparam_df.columns = index

display(dev_hyperparam_df)

100%|██████████| 126/126 [00:02<00:00, 57.00hyperparam/s]


agg                                                                                       macro  \
metric                                                                                      muc   
score                                                                                         p   
func                                                                                       mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons              
addsays    bc    all     False          False                 False                   60.863333   
                         True           False                 False                   61.063333   
                 person  False          False                 False                   87.713333   
                         True           False                 False                   87.630000   
                 speaker False          False                 False                   85.530000   
...                                                                                         ...   
none       wb    all     True           False                 False                   62.293333   
                 person  False          False                 False                   88.063333   
                         True           False                 False                   87.833333   
                 speaker False          False                 False                   85.673333   
                         True           False                 False                   85.863333   

agg                                                                                             \
metric                                                                                           
score                                                                                            
func                                                                                       std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons             
addsays    bc    all     False          False                 False                   4.300492   
                         True           False                 False                   4.503791   
                 person  False          False                 False                   3.910298   
                         True           False                 False                   3.789076   
                 speaker False          False                 False                   2.982901   
...                                                                                        ...   
none       wb    all     True           False                 False                   4.298399   
                 person  False          False                 False                   3.981574   
                         True           False                 False                   3.747323   
                 speaker False          False                 False                   3.298429   
                         True           False                 False                   3.436064   

agg                                                                                              \
metric                                                                                            
score                                                                                         r   
func                                                                                       mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons              
addsays    bc    all     False          False                 False                   87.596667   
                         True           False                 False                   87.873333   
                 person  False          False                 False                   74.453333   
                         True           False         

In [99]:
# best baseline performance on dev set
dev_hyperparam_df[dev_hyperparam_df["macro"]["conll"]["f"]["mean"] == (
                        dev_hyperparam_df["macro"]["conll"]["f"]["mean"].max())]

agg                                                                                   macro  \
metric                                                                                  muc   
score                                                                                     p   
func                                                                                   mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons          
addsays    wb    speaker True           False                 False                   86.26   

agg                                                                                            \
metric                                                                                          
score                                                                                           
func                                                                                      std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons            
addsays    wb    speaker True           False                 False                   3.15073   

agg                                                                                              \
metric                                                                                            
score                                                                                         r   
func                                                                                       mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons              
addsays    wb    speaker True           False                 False                   80.753333   

agg                                                                                             \
metric                                                                                           
score                                                                                            
func                                                                                       std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons             
addsays    wb    speaker True           False                 False                   2.638301   

agg                                                                                              \
metric                                                                                            
score                                                                                         f   
func                                                                                       mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons              
addsays    wb    speaker True           False                 False                   83.393333   

agg                                                                                             \
metric                                                                                           
score                                                                                            
func                                                                                       std   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singletons             
addsays    wb    speaker True           False                 False                   2.273551   

agg                                                                                              \
metric                                                                                     bcub   
score                                                                                         p   
func                                                                                       mean   
preprocess genre entity  merge_speakers provide_gold_mentions remove_gold_singleto

The baseline achieves 62.68 conll F1 on dev set

In [100]:
# Find how baseline performance varies with preprocess, entity, merge_speakers, split_len, overlap_len, merge_strategy
hyperparams = ["preprocess", "entity", "merge_speakers", "split_len", "overlap_len", "merge_strategy"]
for hyperparam in hyperparams:
    # Create paired data for t-test
    levels = sorted(train_hyperparam_df.index.get_level_values(hyperparam).unique().astype("<U18").tolist())
    print(f"{hyperparam:15s}: {levels}")
    group_hyperparams = hyperparams.copy()
    group_hyperparams.remove(hyperparam)
    paired_values = []
    for _, df in train_hyperparam_df.groupby(group_hyperparams, axis=0):
        assert len(df) == len(levels)
        df.sort_values(hyperparam, inplace=True)
        paired_values.append(tuple(df["macro"]["conll"]["f"]["mean"].values.tolist()))
    paired = np.array(paired_values)
    means = np.round(np.mean(paired, axis=0), 3).tolist()
    print(f"{'means':15s}: {means}")

    # Conduct t-tests and correct p-value
    n_comparisons = len(levels)*(len(levels) - 1)/2
    for i in range(len(levels)):
        for j in range(i + 1, len(levels)):
            statistic = scipy.stats.ttest_rel(paired[:, i], paired[:, j], alternative="two-sided")
            print(f"\t{levels[i]:12s} (\u03bc = {np.mean(paired[:, i]):.2f}) vs {levels[j]:12s} "
                  f"(\u03bc = {np.mean(paired[:, j]):.2f}): pvalue = {n_comparisons * statistic.pvalue:.2e} "
                  f"statistic = {statistic.statistic:.3f}")
    print()

preprocess     : ['addsays', 'nocharacters', 'none']
means          : [49.315, 29.424, 48.062]
	addsays      (μ = 49.31) vs nocharacters (μ = 29.42): pvalue = 5.97e-64 statistic = 20.221
	addsays      (μ = 49.31) vs none         (μ = 48.06): pvalue = 8.65e-55 statistic = 18.187
	nocharacters (μ = 29.42) vs none         (μ = 48.06): pvalue = 2.85e-60 statistic = -19.406

entity         : ['all', 'person', 'speaker']
means          : [40.219, 46.252, 40.33]
	all          (μ = 40.22) vs person       (μ = 46.25): pvalue = 3.60e-158 statistic = -43.183
	all          (μ = 40.22) vs speaker      (μ = 40.33): pvalue = 2.73e+00 statistic = -0.113
	person       (μ = 46.25) vs speaker      (μ = 40.33): pvalue = 1.97e-09 statistic = 6.319

merge_speakers : ['False', 'True']
means          : [39.956, 44.578]
	False        (μ = 39.96) vs True         (μ = 44.58): pvalue = 1.70e-115 statistic = -28.341

split_len      : ['2048', '3072', '4096', '5120']
means          : [40.372, 41.855, 42.828, 44.012

In [101]:
# Find how baseline performance varies with preprocess, entity, and merge_speakers for dev set
hyperparams = ["preprocess", "entity", "genre", "merge_speakers"]
for hyperparam in hyperparams:
    # Create paired data for t-test
    levels = sorted(dev_hyperparam_df.index.get_level_values(hyperparam).unique().astype("<U18").tolist())
    print(f"{hyperparam:15s}: {levels}")
    group_hyperparams = hyperparams.copy()
    group_hyperparams.remove(hyperparam)
    paired_values = []
    for _, df in dev_hyperparam_df.groupby(group_hyperparams, axis=0):
        assert len(df) == len(levels)
        df.sort_values(hyperparam, inplace=True)
        paired_values.append(tuple(df["macro"]["conll"]["f"]["mean"].values.tolist()))
    paired = np.array(paired_values)
    means = np.round(np.mean(paired, axis=0), 3).tolist()
    print(f"{'means':15s}: {means}")

    # Conduct t-tests and correct p-value
    n_comparisons = len(levels)*(len(levels) - 1)/2
    for i in range(len(levels)):
        for j in range(i + 1, len(levels)):
            statistic = scipy.stats.ttest_rel(paired[:, i], paired[:, j], alternative="two-sided")
            print(f"\t{levels[i]:12s} (\u03bc = {np.mean(paired[:, i]):.2f}) vs {levels[j]:12s} "
                  f"(\u03bc = {np.mean(paired[:, j]):.2f}): pvalue = {n_comparisons * statistic.pvalue:.2e} "
                  f"statistic = {statistic.statistic:.3f}")
    print()

preprocess     : ['addsays', 'nocharacters', 'none']
means          : [55.119, 38.921, 54.702]
	addsays      (μ = 55.12) vs nocharacters (μ = 38.92): pvalue = 2.10e-07 statistic = 6.556
	addsays      (μ = 55.12) vs none         (μ = 54.70): pvalue = 3.56e-06 statistic = 5.692
	nocharacters (μ = 38.92) vs none         (μ = 54.70): pvalue = 3.27e-07 statistic = -6.421

entity         : ['all', 'person', 'speaker']
means          : [45.694, 55.193, 47.855]
	all          (μ = 45.69) vs person       (μ = 55.19): pvalue = 7.32e-52 statistic = -110.253
	all          (μ = 45.69) vs speaker      (μ = 47.86): pvalue = 1.16e+00 statistic = -0.878
	person       (μ = 55.19) vs speaker      (μ = 47.86): pvalue = 1.47e-02 statistic = 2.975

genre          : ['bc', 'bn', 'mz', 'nw', 'pt', 'tc', 'wb']
means          : [49.631, 49.639, 49.492, 49.572, 49.595, 49.665, 49.472]
	bc           (μ = 49.63) vs bn           (μ = 49.64): pvalue = 1.92e+01 statistic = -0.112
	bc           (μ = 49.63) vs mz       

In [103]:
for _, df in dev_hyperparam_df.groupby(["preprocess", "genre", "merge_speakers"], axis=0):
    display(df.loc[:, ("macro", "conll", "f", "mean")])

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
addsays     bc     all      False           False                  False                     47.240000
                   person   False           False                  False                     57.530000
                   speaker  False           False                  False                     60.761111
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
addsays     bc     all      True            False                  False                     47.083333
                   person   True            False                  False                     56.906667
                   speaker  True            False                  False                     60.614444
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
addsays     bn     all      False           False                  False                     47.340000
                   person   False           False                  False                     57.473333
                   speaker  False           False                  False                     60.704444
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
addsays     bn     all      True            False                  False                     47.186667
                   person   True            False                  False                     56.848889
                   speaker  True            False                  False                     60.558889
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
addsays     mz     all      False           False                  False                     47.793333
                   person   False           False                  False                     56.348889
                   speaker  False           False                  False                     60.926667
Name: (macro, conll, f, mean), dtype: float64

...

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
none        pt     all      True            False                  False                     46.724444
                   person   True            False                  False                     56.502222
                   speaker  True            False                  False                     60.122222
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
none        tc     all      False           False                  False                     47.531111
                   person   False           False                  False                     56.494444
                   speaker  False           False                  False                     60.691111
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
none        tc     all      True            False                  False                     47.096667
                   person   True            False                  False                     56.047778
                   speaker  True            False                  False                     60.431111
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
none        wb     all      False           False                  False                     47.617778
                   person   False           False                  False                     56.518889
                   speaker  False           False                  False                     60.183333
Name: (macro, conll, f, mean), dtype: float64

preprocess  genre  entity   merge_speakers  provide_gold_mentions  remove_gold_singletons
none        wb     all      True            False                  False                     46.808889
                   person   True            False                  False                     56.511111
                   speaker  True            False                  False                     59.747778
Name: (macro, conll, f, mean), dtype: float64